# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

pd.options.mode.chained_assignment = None  # default='warn'

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load the data from csv
csv_file = "output_data/cities.csv"

weather_df = pd.read_csv(csv_file)
weather_df


,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
0,NZ,Christchurch,-43.5333,172.6333,1614229429,62.64,67,90,8.05
1,AU,Busselton,-33.6500,115.3333,1614229430,79.68,46,4,6.46
2,RU,Muravlenko,63.7898,74.5230,1614229431,-28.03,85,7,6.29
3,US,Hilo,19.7297,-155.0900,1614229219,67.87,78,75,6.91
4,RU,Provideniya,64.3833,-173.3000,1614229434,5.00,72,0,13.42
...,...,...,...,...,...,...,...,...,...
566,US,Kahului,20.8947,-156.4700,1614230276,69.49,64,40,20.71
567,BR,Pitimbu,-7.4706,-34.8086,1614230278,78.80,94,75,3.44
568,CL,Coyhaique,-45.5752,-72.0662,1614230279,50.67,98,17,3.15
569,CN,Yunyang,33.4474,112.7142,1614230281,48.00,39,99,15.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# gmaps config
gmaps.configure(api_key = g_key)

# Use the lat and Lon columns as the locations variable
locations = weather_df[["Lat", "Lon"]]

# Use the humidity % column of the df as the weight variable
humidity = weather_df["Humidities %"]
humidity

0      67
1      46
2      85
3      78
4      72
       ..
566    64
567    94
568    98
569    39
570    94
Name: Humidities %, Length: 571, dtype: int64

In [4]:
#heatmap
fig = gmaps.figure(center=(25, 15), zoom_level = 1.8)

# Heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1.5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Drop null values
ideal = weather_df.dropna(how='all')
ideal.tail()

,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
566,US,Kahului,20.8947,-156.4700,1614230276,69.49,64,40,20.71
567,BR,Pitimbu,-7.4706,-34.8086,1614230278,78.80,94,75,3.44
568,CL,Coyhaique,-45.5752,-72.0662,1614230279,50.67,98,17,3.15
569,CN,Yunyang,33.4474,112.7142,1614230281,48.00,39,99,15.14
570,BR,Careiro da Várzea,-3.2003,-59.8119,1614230282,77.05,94,0,1.14


In [6]:
ideal.tail()

,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
566,US,Kahului,20.8947,-156.4700,1614230276,69.49,64,40,20.71
567,BR,Pitimbu,-7.4706,-34.8086,1614230278,78.80,94,75,3.44
568,CL,Coyhaique,-45.5752,-72.0662,1614230279,50.67,98,17,3.15
569,CN,Yunyang,33.4474,112.7142,1614230281,48.00,39,99,15.14
570,BR,Careiro da Várzea,-3.2003,-59.8119,1614230282,77.05,94,0,1.14


In [9]:
#I notice the temp needs to some converstions but i wont change it now. I will just use the data the way it is for now. 
ideal = ideal.loc[(ideal["Max Temp (F)"]) < 85 & (ideal["Max Temp (F)"] > 70)] 
ideal

# narrow the list by selecting ideal wind speeds 
wind_speed = ideal.loc[(ideal["Wind Speed"] < 10)]
wind_speed

#narrow the humiditie levels
new_ideal_df = wind_speed.loc[(wind_speed["Humidities %"] < 80)]
new_ideal_df

,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
33,RU,Chara,56.9064,118.2631,1614229476,-6.56,79,87,1.83
141,CA,Thompson,55.7435,-97.8558,1614229453,-14.80,76,20,1.16
181,CA,Tuktoyaktuk,69.4541,-133.0374,1614229696,-40.00,67,1,6.91
234,CA,Inuvik,68.3499,-133.7218,1614229774,-31.88,68,0,8.05
287,CA,Norman Wells,65.2820,-126.8329,1614229858,-5.80,77,90,3.44
410,RU,Vysokaya Gora,55.9128,49.3017,1614230048,-13.00,70,0,8.95
425,CA,Moose Factory,51.2585,-80.6094,1614230071,-4.92,71,20,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = new_ideal_df
hotel_df

,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed
33,RU,Chara,56.9064,118.2631,1614229476,-6.56,79,87,1.83
141,CA,Thompson,55.7435,-97.8558,1614229453,-14.80,76,20,1.16
181,CA,Tuktoyaktuk,69.4541,-133.0374,1614229696,-40.00,67,1,6.91
234,CA,Inuvik,68.3499,-133.7218,1614229774,-31.88,68,0,8.05
287,CA,Norman Wells,65.2820,-126.8329,1614229858,-5.80,77,90,3.44
410,RU,Vysokaya Gora,55.9128,49.3017,1614230048,-13.00,70,0,8.95
425,CA,Moose Factory,51.2585,-80.6094,1614230071,-4.92,71,20,5.75


In [11]:
#create a new colum of hotel names
hotel_df["Hotel Names"] = ""
hotel_df

,Country,City,Lat,Lon,Date,Max Temp (F),Humidities %,Cloudinesses,Wind Speed,Hotel Names
33,RU,Chara,56.9064,118.2631,1614229476,-6.56,79,87,1.83,
141,CA,Thompson,55.7435,-97.8558,1614229453,-14.80,76,20,1.16,
181,CA,Tuktoyaktuk,69.4541,-133.0374,1614229696,-40.00,67,1,6.91,
234,CA,Inuvik,68.3499,-133.7218,1614229774,-31.88,68,0,8.05,
287,CA,Norman Wells,65.2820,-126.8329,1614229858,-5.80,77,90,3.44,
410,RU,Vysokaya Gora,55.9128,49.3017,1614230048,-13.00,70,0,8.95,
425,CA,Moose Factory,51.2585,-80.6094,1614230071,-4.92,71,20,5.75,


In [12]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [17]:
# api request the hotels within the param 
for index, row in hotel_df.iterrows():
    
    # lat and lon from df
    lat = row["Lat"]
    lon = row["Lon"]

    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results by {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    #print(json.dumps(response, indent=4, sort_keys=True))
    
    #results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing/skipping...........")
        
    print("---------------------------------------")
print(hotel_df)

Retrieving Results by 33: Chara.
Closest hotel is Sberbank Rossii.
---------------------------------------
Retrieving Results by 141: Thompson.
Closest hotel is Best Western Thompson Hotel & Suites.
---------------------------------------
Retrieving Results by 181: Tuktoyaktuk.
Closest hotel is Hunter's B&B.
---------------------------------------
Retrieving Results by 234: Inuvik.
Closest hotel is Nova Inn Inuvik.
---------------------------------------
Retrieving Results by 287: Norman Wells.
Closest hotel is Heritage Hotel.
---------------------------------------
Retrieving Results by 410: Vysokaya Gora.
Missing/skipping...........
---------------------------------------
Retrieving Results by 425: Moose Factory.
Closest hotel is Super 8 by Wyndham Moosonee.
---------------------------------------
    Country           City      Lat       Lon        Date  Max Temp (F)  \
33       RU          Chara  56.9064  118.2631  1614229476         -6.56   
141      CA       Thompson  55.7435  -9

In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add a layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))